In [ ]:
!pip install langchain
!pip install pypdf
!pip install openai

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install faiss-cpu


In [ ]:
!pip install tiktoken

In [ ]:
! pip install chromadb

# Load Documents

In [ ]:
import os
import openai
import sys
# sys.path.append('../..')

# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv()) # read local .env file


os.environ['OPENAI_API_KEY']=''

In [ ]:
from langchain.document_loaders import PyPDFLoader

# Load PDF
loaders = [

    PyPDFLoader("/content/drive/MyDrive/Mercedes/mercedes-benz-annual-report-2022-incl-combined-management-report-mbg-ag-201-283.pdf"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

# Split document into Chunks

In [55]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(
    chunk_size = 500,
    chunk_overlap =150
)
#text_splitter[0]

In [ ]:
splits = text_splitter.split_documents(docs)

In [ ]:
import langchain
langchain.debug = False

#langchain.debug=True

In [ ]:
len(splits)

89

In [ ]:
splits[0]

Document(page_content='Annual Report 2022', metadata={'source': '/content/mercedes-benz-annual-report-reduced.pdf', 'page': 0})

In [ ]:
splits[5]

# Initiate embeddings

In [54]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

# Vector Store

Chroma

In [ ]:
from langchain.vectorstores import Chroma

In [ ]:
persist_directory = '/chroma/'
!rm -rf ./docs/chroma
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

# Similarity Search (Explanation)

In [ ]:
# question = "Who is Chairman of the Supervisory Board of Mercedes-Benz Group AG"
# docs = vectordb.similarity_search(question,k=3)
# len(docs)

3

In [ ]:
docs[0]

In [ ]:
docs[6]

In [ ]:
def get_context(question):
    docs = vectordb1.similarity_search(question,k=5)
    return docs

In [ ]:
get_context("Any mergers?")

[link text](https://)# LLM model initialization

In [56]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.1)
# llm.predict("what is ccsp")

In [ ]:
def Query_from_user(qa_chain,questions):
  print(questions)
  for question in questions:
    result = qa_chain({"query": question})
    return result["result"]


Chromadb


In [ ]:
from langchain.prompts import PromptTemplate
template = """Use the following pieces of context to answer the question at the end.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)

# Run chain
from langchain.chains import RetrievalQA
question = "Any mergers?"
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(search_kwargs={"k": 5}),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})

Query_from_user(qa_chain,question)


In [ ]:
import csv

#Save the answers in a CSV file

In [59]:
import csv
from langchain.prompts import PromptTemplate
template = """Use the following pieces of context to answer the question at the end.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)

# Run chain
from langchain.chains import RetrievalQA
question = [str(question) for question in input("Enter your queries: ").split('?')]
#print(question)
qa_chain_multi = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(search_kwargs={"k": 5}),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})


#Query_from_user(qa_chain_multi,question)

Enter your queries: - Any mergers?  - Any Acquisitions?


In [ ]:
with open("questions_and_answers_141_200_NEW.csv", "w", newline="") as file:  # Create file if it doesn't exist
        csv_writer = csv.writer(file)
        csv_writer.writerow(["Question", "Answer"])


        for que in question:
            result = qa_chain_multi({"query": que})
            answer = result["result"]
            csv_writer.writerow([que, answer])

        # Write the question and answer as a row in the CSV file



In [ ]:
question

['Any Acquisitions', '']

In [ ]:
def input_questions():
  #to handle the input and send it to the above cell


SyntaxError: incomplete input (<ipython-input-91-d4b168fe4dc4>, line 1)

FAISS

In [ ]:
from langchain.vectorstores import FAISS
vectordb1 = FAISS.from_documents(splits,embedding)

In [ ]:

# Build prompt
from langchain.prompts import PromptTemplate
template = """Use the following pieces of context to answer the question at the end.
{context}

%questions%
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)

# Run chain
from langchain.chains import RetrievalQA
question_f = "Give all the chairmans mentioned in the report, which company report is this"
qa_chain_f = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb1.as_retriever(search_kwargs={"k": 5}),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})

Query_from_user(qa_chain_f,question_f)
